In [1]:
import tensorflow as tf
import ray
from ray.tune.registry import register_env
from ray.rllib.algorithms import dqn, ppo

from env import *
from model import *
from action_distribution import *

In [2]:
%load_ext autoreload
%autoreload 2

tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



True

In [3]:
if ray.is_initialized():
    ray.shutdown()
ray.init()

2023-03-13 20:26:58,071	INFO worker.py:1553 -- Started a local Ray instance.


Python version:,3.10.9
Ray version:,2.3.0


In [4]:
n_disks = 3
env_config = {
    'n_disks': n_disks,
    'max_n_steps': 10000 #2 ** (n_disks + 2)
}

ENV_NAME = 'hanoi_tower'
register_env(ENV_NAME, create_env)

config = ppo.PPOConfig()
config = config.training(lr=1e-3,
                         gamma=0.9,
                         train_batch_size=512 * 2,
                         sgd_minibatch_size=256 * 1,
                         num_sgd_iter=16 * 1)
config = config.rollouts(num_rollout_workers=8,
                         num_envs_per_worker=4)
config = config.environment(ENV_NAME, 
                            env_config=env_config)
config = config.resources(num_gpus=0)
config = config.framework('tf')

model_config = {
    'n_hiddens': 256
}

config.model.update({
    'custom_model': MODEL_NAME,
    'custom_model_config': model_config,
    'custom_action_dist': ACTION_DIST_NAME,
    'vf_share_layers': True
})

config.seed = 19
agent = config.build()

2023-03-13 20:26:58,663	INFO algorithm_config.py:2899 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
E0313 20:26:58.676466000 8688435520 fork_posix.cc:76]                  Other threads are currently calling into gRPC, skipping fork() handlers
2023-03-13 20:26:58,683	INFO algorithm.py:506 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=3846) Metal device set to: Apple M1
(RolloutWorker pid=3846) 
(RolloutWorker pid=3846) systemMemory: 16.00 GB
(RolloutWorker pid=3846) maxCacheSize: 5.33 GB
(RolloutWorker pid=3846) 
(RolloutWorker pid=3841) Metal device set to: Apple M1
(RolloutWorker pid=3841) 
(RolloutWorker pid=3841) systemMemory: 16.00 GB
(RolloutWorker pid=3841) maxCacheSize: 5.33 GB
(RolloutWorker pid=3841) 
(RolloutWorker pid=3846) Model: "base_model"
(RolloutWorker pid=3846) _________________________________________________________________
(RolloutWorker pid=3846)  Layer (type)                Output Shape              Param #   
(RolloutWorker pid=3846) =================================================================
(RolloutWorker pid=3846)  obs_input (InputLayer)      [(None, 3, 3)]            0         
(RolloutWorker pid=3846)                                                                  
(RolloutWorker pid=3846)  flatten (Flatten)           (None, 9)                 0         
(Roll

(RolloutWorker pid=3840) 2023-03-13 20:27:02,904	WARNING env.py:156 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(RolloutWorker pid=3840) 2023-03-13 20:27:02,904	WARNING env.py:166 -- Your env reset() method appears to take 'seed' or 'return_info' arguments. Note that these are not yet supported in RLlib. Seeding will take place using 'env.seed()' and the info dict will not be returned from reset.


(RolloutWorker pid=3840) Model: "base_model"
(RolloutWorker pid=3840) _________________________________________________________________
(RolloutWorker pid=3840)  Layer (type)                Output Shape              Param #   
(RolloutWorker pid=3840) =================================================================
(RolloutWorker pid=3840)  obs_input (InputLayer)      [(None, 3, 3)]            0         
(RolloutWorker pid=3840)                                                                  
(RolloutWorker pid=3840)  flatten (Flatten)           (None, 9)                 0         
(RolloutWorker pid=3840)                                                                  
(RolloutWorker pid=3840)  context1 (Dense)            (None, 256)               2560      
(RolloutWorker pid=3840)                                                                  
(RolloutWorker pid=3840)  context2 (Dense)            (None, 256)               65792     
(RolloutWorker pid=3840)                     

2023-03-13 20:27:05,301	WARNING util.py:67 -- Install gputil for GPU system monitoring.


In [5]:
config.to_dict()

{'extra_python_environs_for_driver': {},
 'extra_python_environs_for_worker': {},
 'num_gpus': 0,
 'num_cpus_per_worker': 1,
 'num_gpus_per_worker': 0,
 '_fake_gpus': False,
 'num_trainer_workers': 0,
 'num_gpus_per_trainer_worker': 0,
 'num_cpus_per_trainer_worker': 1,
 'custom_resources_per_worker': {},
 'placement_strategy': 'PACK',
 'eager_tracing': False,
 'eager_max_retraces': 20,
 'tf_session_args': {'intra_op_parallelism_threads': 2,
  'inter_op_parallelism_threads': 2,
  'gpu_options': {'allow_growth': True},
  'log_device_placement': False,
  'device_count': {'CPU': 1},
  'allow_soft_placement': True},
 'local_tf_session_args': {'intra_op_parallelism_threads': 8,
  'inter_op_parallelism_threads': 8},
 'env': 'hanoi_tower',
 'env_config': {'n_disks': 3, 'max_n_steps': 10000},
 'observation_space': None,
 'action_space': None,
 'env_task_fn': None,
 'render_env': False,
 'clip_rewards': None,
 'normalize_actions': True,
 'clip_actions': False,
 'disable_env_checking': False,
 '

In [8]:
for _ in range(50):
    res = agent.train()

KeyboardInterrupt: 

In [7]:
env = create_env(env_config)
obs, _ = env.reset()

episode_reward = 0
done = False
while not done:
    action = agent.compute_single_action(obs, 
                                         explore=False)
    print(action)
    obs, reward, done, terminated, info = env.step(action)
    episode_reward += reward


[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0

KeyboardInterrupt: 

In [11]:
obs

array([[0, 0, 0],
       [0, 0, 0],
       [1, 2, 3]], dtype=int32)

In [12]:
episode_reward

-7.0

In [ ]:
env.observation_space

Box(0, 2, (3, 2), int32)

In [5]:
env = HanoiTower(config=env_config)

In [6]:
env.action_space.sample()

array([2, 1])

In [ ]:
obs = env.reset()
env.observation_space.contains(obs)

In [ ]:
obs

In [ ]:
env.step([0, 1])